In [6]:
pip install optuna 

Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.model_selection import train_test_split
import optuna 
from sklearn.preprocessing import StandardScaler
import pandas as pd 

In [8]:
df=pd.read_csv("C:\\Users\\DELL\\Downloads\\diabetes.csv")

In [9]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
## problem there is some missing value which is represented bu 0 in insulin,and mostly some more columns  

In [11]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [12]:
import numpy as np 

In [13]:
## so what we can do is repace 0 with mean of that column 
columns_null = ["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]
df[columns_null]=df[columns_null].replace(0,np.nan)
df[columns_null]=df[columns_null].fillna(df[columns_null].mean())


                                  

## split into x and y 
make train and test split 
standard scaler

In [14]:
X=df.drop(columns=["Outcome"],axis=1)
y=df["Outcome"]

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [16]:
scale=StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

In [17]:
X_train.shape

(614, 8)

In [18]:
X_test.shape

(154, 8)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [20]:
from sklearn.model_selection import cross_val_score

In [21]:
def objective_class(trial):## hum ek objective function banayenge jisme hum hyperparameter tuning karenge 
    n_estimators=trial.suggest_int("n_estimators",50,250)
    max_depth=trial.suggest_int("max_depth",2,20)
    ## define our model 
    model=RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    ## perform 3 cross validation 
    score=cross_val_score(model,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score

In [22]:
## make a study object 
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(objective_class,n_trials=50)

[I 2025-12-06 14:55:23,333] A new study created in memory with name: no-name-8144d518-887b-44ef-9123-1e2455a1b76e
[I 2025-12-06 14:55:25,980] Trial 0 finished with value: 0.771975131516021 and parameters: {'n_estimators': 125, 'max_depth': 6}. Best is trial 0 with value: 0.771975131516021.
[I 2025-12-06 14:55:30,498] Trial 1 finished with value: 0.7703331739199745 and parameters: {'n_estimators': 147, 'max_depth': 5}. Best is trial 0 with value: 0.771975131516021.
[I 2025-12-06 14:55:35,146] Trial 2 finished with value: 0.7736091184441256 and parameters: {'n_estimators': 202, 'max_depth': 8}. Best is trial 2 with value: 0.7736091184441256.
[I 2025-12-06 14:55:39,177] Trial 3 finished with value: 0.7638291088793241 and parameters: {'n_estimators': 227, 'max_depth': 4}. Best is trial 2 with value: 0.7736091184441256.
[I 2025-12-06 14:55:41,296] Trial 4 finished with value: 0.7735931771082417 and parameters: {'n_estimators': 140, 'max_depth': 18}. Best is trial 2 with value: 0.77360911844

In [23]:
study.best_trial.values

[0.7866411605292524]

In [24]:
study.best_trial.params

{'n_estimators': 120, 'max_depth': 13}

In [25]:
best_model=RandomForestClassifier(** study.best_trial.params,random_state=42)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
accuracy_score(y_test,y_pred)


0.7597402597402597

In [26]:
## we have acheived accuracy of 75 percent 

In [27]:
## now lets use some diff sampler 
def objective_class(trial):
    n_estimators=trial.suggest_int("n_estimators",50,250)
    max_depth=trial.suggest_int("max_depth",2,20)
    model=RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    score=cross_val_score(model,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score

In [28]:
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.RandomSampler())
study.optimize(objective_class,n_trials=50)

[I 2025-12-06 14:57:23,823] A new study created in memory with name: no-name-16420169-2270-4761-89d6-fcf39743923d
[I 2025-12-06 14:57:25,025] Trial 0 finished with value: 0.7507890961262554 and parameters: {'n_estimators': 66, 'max_depth': 4}. Best is trial 0 with value: 0.7507890961262554.
[I 2025-12-06 14:57:27,528] Trial 1 finished with value: 0.7833811573409851 and parameters: {'n_estimators': 129, 'max_depth': 13}. Best is trial 1 with value: 0.7833811573409851.
[I 2025-12-06 14:57:29,750] Trial 2 finished with value: 0.7556830862426271 and parameters: {'n_estimators': 125, 'max_depth': 4}. Best is trial 1 with value: 0.7833811573409851.
[I 2025-12-06 14:57:32,091] Trial 3 finished with value: 0.744300972421489 and parameters: {'n_estimators': 225, 'max_depth': 2}. Best is trial 1 with value: 0.7833811573409851.
[I 2025-12-06 14:57:34,347] Trial 4 finished with value: 0.7768691216323927 and parameters: {'n_estimators': 132, 'max_depth': 11}. Best is trial 1 with value: 0.783381157

In [29]:
study.best_trial.values

[0.7833811573409851]

In [30]:
study.best_trial.params

{'n_estimators': 129, 'max_depth': 13}

In [31]:
best_model=RandomForestClassifier(n_estimators=146,max_depth=13,random_state=42)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.7532467532467533

In [32]:
from sklearn import svm
from sklearn.svm import SVC


In [33]:
def function(trial):
    category=trial.suggest_categorical("category",["SVM","RandomForest"])
    if category=="SVM":
        c=trial.suggest_float("C",0.1,100,log=True)
        gamma=trial.suggest_categorical("gamma",["scale","auto"])
        kernel=trial.suggest_categorical("kernel",["linear","rbf","poly"])
        model=SVC(C=c,gamma=gamma,kernel=kernel,random_state=42)
    elif category=="RandomForest":
        n_estimators=trial.suggest_int("n_estimators",50,250)
        max_depth=trial.suggest_int("max_depth",2,20)
        model=RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
    score=cross_val_score(model,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score 

        

In [34]:
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(function,n_trials=50)
                          

[I 2025-12-06 14:59:19,535] A new study created in memory with name: no-name-cd94b664-8428-4e9e-8b17-5ea265ecd735
[I 2025-12-06 14:59:19,630] Trial 0 finished with value: 0.7703411445879165 and parameters: {'category': 'SVM', 'C': 0.16550474228523546, 'gamma': 'scale', 'kernel': 'linear'}. Best is trial 0 with value: 0.7703411445879165.
[I 2025-12-06 14:59:20,714] Trial 1 finished with value: 0.7719910728519049 and parameters: {'category': 'RandomForest', 'n_estimators': 91, 'max_depth': 11}. Best is trial 1 with value: 0.7719910728519049.
[I 2025-12-06 14:59:20,784] Trial 2 finished with value: 0.7280089271480951 and parameters: {'category': 'SVM', 'C': 1.3481758469065164, 'gamma': 'scale', 'kernel': 'poly'}. Best is trial 1 with value: 0.7719910728519049.
[I 2025-12-06 14:59:21,674] Trial 3 finished with value: 0.7768531802965088 and parameters: {'category': 'RandomForest', 'n_estimators': 82, 'max_depth': 6}. Best is trial 3 with value: 0.7768531802965088.
[I 2025-12-06 14:59:24,552

In [35]:
best_trial=study.best_trial
best_trial.values

[0.7882751474573569]

## ended up with a accuracy 0f 79 percent 4 percent increase 

In [36]:
best_trial.params

{'category': 'RandomForest', 'n_estimators': 122, 'max_depth': 13}

In [37]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_category,params_gamma,params_kernel,params_max_depth,params_n_estimators,state
0,0,0.770341,2025-12-06 14:59:19.539000,2025-12-06 14:59:19.630782,0 days 00:00:00.091782,0.165505,SVM,scale,linear,NaN,NaN,COMPLETE
1,1,0.771991,2025-12-06 14:59:19.633691,2025-12-06 14:59:20.714940,0 days 00:00:01.081249,NaN,RandomForest,NaN,NaN,11.0,91.0,COMPLETE
2,2,0.728009,2025-12-06 14:59:20.721114,2025-12-06 14:59:20.783220,0 days 00:00:00.062106,1.348176,SVM,scale,poly,NaN,NaN,COMPLETE
3,3,0.776853,2025-12-06 14:59:20.784914,2025-12-06 14:59:21.674917,0 days 00:00:00.890003,NaN,RandomForest,NaN,NaN,6.0,82.0,COMPLETE
4,4,0.758951,2025-12-06 14:59:21.674917,2025-12-06 14:59:24.552044,0 days 00:00:02.877127,NaN,RandomForest,NaN,NaN,4.0,163.0,COMPLETE
5,5,0.767073,2025-12-06 14:59:24.559279,2025-12-06 14:59:24.669051,0 days 00:00:00.109772,0.230331,SVM,auto,linear,NaN,NaN,COMPLETE
6,6,0.727993,2025-12-06 14:59:24.675976,2025-12-06 14:59:24.831544,0 days 00:00:00.155568,6.274874,SVM,scale,poly,NaN,NaN,COMPLETE
7,7,0.700351,2025-12-06 14:59:24.831544,2025-12-06 14:59:25.000370,0 days 00:00:00.168826,56.045112,SVM,auto,rbf,NaN,NaN,COMPLETE
8,8,0.768699,2025-12-06 14:59:25.000370,2025-12-06 14:59:25.131416,0 days 00:00:00.131046,6.358384,SVM,scale,linear,NaN,NaN,COMPLETE
9,9,0.767081,2025-12-06 14:59:25.134302,2025-12-06 14:59:25.183935,0 days 00:00:00.049633,0.282704,SVM,auto,linear,NaN,NaN,COMPLETE
